# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [ ]:
import os

print(os.uname())

# In the cloud environment?
if 'root' in os.environ['HOME']:
    UENVPATH = '/data/'
    !pip -q install /home/workspace/python

# In the standalone environment?
if 'ferenc' in os.environ['HOME']:
    UENVPATH = '/home/ferenc/Python/rl/udadrl/data/'


from unityagents import UnityEnvironment
import numpy as np

import pprint as pp

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [ ]:
# ONE Agent, Standalone
#UENVCHOICE = 'Reacher_Linux/Reacher.x86_64'

# TWENTY Agents, Standalone
#UENVCHOICE = 'Reacher_Linux_20/Reacher.x86_64'

# ONE Agent, Cloud, No-Visuals 
#UENVCHOICE = 'Reacher_One_Linux_NoVis/Reacher_One_Linux_NoVis.x86_64'

# TWENTY Agents, Cloud, No-Visuals 
UENVCHOICE = 'Reacher_Linux_NoVis/Reacher.x86_64'

env = UnityEnvironment( file_name=os.path.join( UENVPATH, UENVCHOICE ) )




Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# FA DEBUG! DEBUG! DEBUG! 
type(env)

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

In [ ]:
# FA  DEBUG! DEBUG! DEBUG! 
print(env.brain_names)
print( type(brain) )
brain.brain_name

In [ ]:
# FA  DEBUG! DEBUG! DEBUG!
print(f'Observation Space: {brain.vector_observation_space_size} dimensions of {brain.vector_observation_space_type} type') 
print(f'Action Space: {brain.vector_action_space_size} dimensions of {brain.vector_observation_space_type} type')

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [ ]:
# Some reminders about lambda functions
# Thx2: https://realpython.com/python-lambda/
# Thx2: https://thispointer.com/python-how-to-use-if-else-elif-in-lambda-functions/


env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)

epc = 0
print(f'We have {num_agents} active agent' + ( lambda x: 's' if x>1 else '' )(num_agents) )

while True:
    epc += 1
    #print(f'\r#{epc}', end = ' ')
    print(f'\r#{str(epc).rjust(6)}', end = ' ')
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    #import pdb; pdb.set_trace() # Debug! Debug! Debug! Debug! Debug! Debug! 
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    rewards = env_info.rewards
    nprewards = np.array(rewards)
    scores += rewards                         # update the score (for each agent)
    if np.sum(nprewards) > 0:
        #print(f'\r#{epc} Rewards = {rewards}')
        maskagent = nprewards > 0
        agentbefore = False
        for (nagent, action) in enumerate(actions):
            if maskagent[nagent]:
                if agentbefore:
                    print('\r#' + '&'.rjust(6), end = ' ')
                print(' -> Agent {:0>2d} got reward {:+.5f} for action: {}'.format(nagent+1, rewards[nagent], action))
                agentbefore = True
                #pp.pprint(list(action))
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('#')
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

### When finished, you can close the environment.
#### Just not for now because unit testing below

In [ ]:
# env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

#### Tracing the error in the 'mya2cnet' module code refactoring below

In [ ]:
# TEST! TEST! TEST! TEST! TEST! TEST! TEST! TEST! TEST! TEST! TEST! TEST! TEST! TEST! 
#import pdb; pdb.set_trace() # Debug! Debug! Debug! Debug! Debug! Debug! 

import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(20200808) # Debug! Debug! Debug! Debug! Debug! Debug! Debug! Debug!

# Format: IN_Num [Layer 1] (OUT_Num = IN_Num) [Layer 2] OUT_Num = ...
HIDDEN_DIMS_DEFAULT = {
    'shared' : (512, 512, 256, 256),
    'actor' : (256, 128, 128, 64),
    'critic' : (256, 128, 128, 64)
}
hidden_dims = HIDDEN_DIMS_DEFAULT

hlayers = dict()

hlayers['shared'] = nn.ModuleList()
hlayers['actor'] = nn.ModuleList()
hlayers['critic'] = nn.ModuleList()




In [ ]:
# Input layer
input_layer = nn.Linear( 33, hidden_dims['shared'][0] )

In [ ]:
input_layer

In [ ]:
# Hidden layers shared
for i in range( len(hidden_dims['shared']) -1 ):
    hlayers['shared'].append( nn.Linear( hidden_dims['shared'][i], hidden_dims['shared'][i+1] ) )

In [ ]:
hlayers['shared']

In [ ]:
# Actor layers
for i in range( len(hidden_dims['actor']) ):
    #import pdb; pdb.set_trace() # Debug! Debug! Debug! Debug! Debug! Debug! Debug! Debug!
    if i == 0:
        hlayers['actor'].append( nn.Linear( hidden_dims['shared'][-1], hidden_dims['actor'][i] ) )
    else:
        # hlayers['actor'].append( nn.Linear( hidden_dims['actor'][i], hidden_dims['actor'][i+1] ERROR !!!
        hlayers['actor'].append( nn.Linear( hidden_dims['actor'][i-1], hidden_dims['actor'][i] ) )
    #print( i, hlayers['actor'] ) # Debug! Debug! Debug! Debug! Debug! Debug! Debug! Debug!
        
actor_out_layer = nn.Linear( hidden_dims['actor'][-1], 4 )

In [ ]:
hlayers['actor']

In [ ]:
actor_out_layer

In [ ]:
#Critic layers
for i in range( len(hidden_dims['critic']) ):
    if i == 0:
        hlayers['critic'].append( nn.Linear( hidden_dims['shared'][-1], hidden_dims['critic'][i] ) )
    else:
        hlayers['critic'].append( nn.Linear( hidden_dims['critic'][i-1], hidden_dims['critic'][i] ) )
critic_out_layer = nn.Linear( hidden_dims['critic'][-1], 1 )

In [ ]:
hlayers['critic']

In [ ]:
critic_out_layer

In [ ]:
# Prevents non Pytorch Tensor Object entering the processing stream
def torch_format(state):
    x = state
    if not isinstance(x, torch.Tensor):
        x = torch.tensor(x, dtype=torch.float32)
        if len(x.size()) == 1:
            x = x.unsqueeze(0)
    return x

In [ ]:
def forward(state):
    check_tensor = lambda x: isinstance(x, torch.Tensor)
    x_act = True 
    x_crit = True

    x = torch_format(state)
    x = F.relu(  input_layer(x) )
    for label in ['shared', 'actor', 'critic']:
        for hlayer in  hlayers[label]:
            if label == 'shared':
                x = F.relu(  hlayer(x) )
            if label == 'actor':
                x_act = F.relu(  hlayer(x_act) )
            if label == 'critic':
                x_crit = F.relu(  hlayer(x_crit) )

        # Thx2: https://discuss.pytorch.org/t/copy-deepcopy-vs-clone/55022
        if ( type(x_act) == bool ):
            x_act = x.clone()  # Create an Inplace copy...
        if ( type(x_crit) == bool ):
            x_crit = x.clone() # ...after processing shared layers

    return  actor_out_layer(x_act),  critic_out_layer(x_crit) 

#### Test if states are propagated through the debug network
And make a list of outputs of two A2C instances

In [ ]:
al = []
bl = []

for (dc, st) in enumerate(states):
    a,b = forward(st)
    al.append(a)
    bl.append(b)
#    print(f'({dc+1}) -> Actor: {a}')
#    print(f'({dc+1}) -> Critic: {b}')
    

#### Test if states are propagated through the imported network

In [ ]:
#from mya2cnet import A2CNetwork
import mya2cnet

# Thx2: https://emacs.stackexchange.com/a/13483
import imp
imp.reload(mya2cnet)

tstnet1 = mya2cnet.A2CNetwork(33, 4)
tstnet2 = mya2cnet.A2CNetwork(33, 4)

In [ ]:
for (dc, st) in enumerate(states):
    #import pdb; pdb.set_trace() # Debug! Debug! Debug! Debug! Debug! Debug! Debug! Debug!
    a1,b1 = tstnet1.forward(st)
    a2,b2 = tstnet2.forward(st)
    print(f'({dc+1}) -> Dist. Actor 1-2: {torch.dist(a1, a2)}'.ljust(50) + ' # ' +\
         f'Dist. Actor 1-Notebook {torch.dist(a1, al[dc])}'.rjust(50))
    print(f'({dc+1}) -> Dist. Critic 1-2: {torch.dist(b1, b2)}'.ljust(50) + ' # ' +\
         f'Dist. Critic 1-Notebook {torch.dist(b1, bl[dc])}'.rjust(50))

In [ ]:
for (dc, st) in enumerate(states):
    print(f'({dc+1}) -> {tstnet1.fullpass(st)}')

In [ ]:
for (dc, st) in enumerate(states):
    #import pdb; pdb.set_trace() # Debug! Debug! Debug! Debug! Debug! Debug! Debug! Debug!
    print(f'({dc+1})-> {tstnet1.select_action(st)}') #, end=' ')

#### Test the agent instance

In [ ]:
import mya2cagent

# Thx2: https://emacs.stackexchange.com/a/13483
import imp
imp.reload(mya2cagent)

agent = mya2cagent.a2cagent(len(env_info.agents), env, brain, max_steps = 100)

In [ ]:
procs = mya2cagent.MultiProcEnv(env, 20200808, len(env_info.agents))

In [ ]:
agent.train(states)

In [ ]:
env.close()